# Powerful Feature Selection with Recursive Feature Elimination (RFE) of Sklearn
## Feature selection based on model performance
<img src='images/unsplash.jpg'></img>
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://unsplash.com/@victoriano?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Victoriano Izquierdo</a>
        on 
        <a href='https://unsplash.com/s/photos/selection?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Unsplash</a>
    </strong>
</figcaption>